In [1]:
import os
import cv2
import json
import torch
import pprint
import numpy as np
from tqdm import tqdm
from loguru import logger
from tabulate import tabulate

from pose.utils import collate_fn, geodesic_distance, relative_pose_error, aggregate_metrics, recall_object, project_points

In [2]:
with open("data/pairs/Onepose-test.json") as f:
    dir_list = json.load(f)
len(dir_list)

10

In [3]:
if os.name == 'unix':
    ROOT_DIR = 'data/onepose/'
elif os.name == 'nt':
    ROOT_DIR = 'e:/datasets/OnePose/test_data/'

In [4]:
res_table = []

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'

num_sample = 3000

net = torch.load('./weights/onepose-6d-3000-2024-01-06-00-05-51-0.2705.pth').to(device)

net.eval()

Mkpts_Reg_Model(
  (embedding): Embedding()
  (transformerlayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
    )
    (linear1): Linear(in_features=76, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=76, bias=True)
    (norm1): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((76,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=76, out_features=76, bias=True)
        )
        (linear1): Linear(in_features=76, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inp

In [6]:
for label_idx, test_dict in enumerate(dir_list):
    logger.info(f"Onepose: {label_idx + 1}/{len(dir_list)}")
    metrics = dict()
    metrics.update({'R_errs':[], 't_errs':[], 'inliers':[], "identifiers":[]})
    sample_data = dir_list[label_idx]["0"][0]
    label = sample_data.split("/")[0]
    name = label.split("-")[1]
    dir_name = os.path.dirname(sample_data)
    FULL_ROOT_DIR = os.path.join(ROOT_DIR, dir_name)
    recall_image, all_image = 0, 0
    for rotation_key, rotation_list in zip(test_dict.keys(), test_dict.values()):
        for pair_idx, pair_name in enumerate(tqdm(rotation_list)):
            all_image = all_image + 1
            base_name = os.path.basename(pair_name)
            idx0_name = base_name.split("-")[0]
            idx1_name = base_name.split("-")[1]
            image0_name = os.path.join(FULL_ROOT_DIR, idx0_name)
            image1_name = os.path.join(FULL_ROOT_DIR.replace("color", "color"), idx1_name)

            K0_path = image0_name.replace("color", "intrin_ba").replace("png", "txt")
            K1_path = image1_name.replace("color", "intrin_ba").replace("png", "txt")
            K0 = np.loadtxt(K0_path)
            K1 = np.loadtxt(K1_path)

            pose0_path = image0_name.replace("color", "poses_ba").replace("png", "txt")
            pose1_path = image1_name.replace("color", "poses_ba").replace("png", "txt")
            pose0 = np.loadtxt(pose0_path)
            pose1 = np.loadtxt(pose1_path)
            if pose0.shape[0] == 3:
                pose0 = np.concatenate([pose0, np.array([[0, 0, 0, 1]])], axis=0)
                pose1 = np.concatenate([pose1, np.array([[0, 0, 0, 1]])], axis=0)

            points_file_path = os.path.join('d:/git_project/POPE/data/onepose-points/', pair_name.split("/")[0])
            pre_bbox_path = os.path.join(points_file_path, "pre_bbox")
            mkpts0_path = os.path.join(points_file_path, "mkpts0")
            mkpts1_path = os.path.join(points_file_path, "mkpts1")
            pre_K_path = os.path.join(points_file_path, "pre_K")
            points_name = pair_name.split("/")[-1]
            pre_bbox_path = os.path.join(pre_bbox_path, f'{points_name}.txt')
            mkpts0_path = os.path.join(mkpts0_path, f'{points_name}.txt')
            mkpts1_path = os.path.join(mkpts1_path, f'{points_name}.txt')
            pre_K_path = os.path.join(pre_K_path, f'{points_name}.txt')

            if not os.path.exists(pre_bbox_path):
                continue
            pre_bbox = np.loadtxt(pre_bbox_path)
            mkpts0 = np.loadtxt(mkpts0_path)
            mkpts1 = np.loadtxt(mkpts1_path)
            pre_K = np.loadtxt(pre_K_path)

            if mkpts0.shape[0] > num_sample:
                rand_idx = np.random.choice(mkpts0.shape[0], num_sample, replace=False)
                mkpts0 = mkpts0[rand_idx]
                mkpts1 = mkpts1[rand_idx]
            else:
                mkpts0 = np.concatenate([mkpts0, np.zeros((num_sample - mkpts0.shape[0], 2))], axis=0)
                mkpts1 = np.concatenate([mkpts1, np.zeros((num_sample - mkpts1.shape[0], 2))], axis=0)

            _3d_bbox = np.loadtxt(f"{os.path.join(ROOT_DIR, label)}/box3d_corners.txt")
            bbox_pts_3d, _ = project_points(_3d_bbox, pose1[:3, :4], K1)
            bbox_pts_3d = bbox_pts_3d.astype(np.int32)
            x0, y0, w, h = cv2.boundingRect(bbox_pts_3d)
            x1, y1 = x0 + w, y0 + h
            gt_bbox = np.array([x0, y0, x1, y1])
            is_recalled = recall_object(pre_bbox, gt_bbox)
            recall_image = recall_image + int(is_recalled > 0.5)

            batch_mkpts0 = torch.from_numpy(mkpts0).unsqueeze(0).float().to(device)
            batch_mkpts1 = torch.from_numpy(mkpts1).unsqueeze(0).float().to(device)
            pre_t, pre_rot = net(batch_mkpts0, batch_mkpts1)
            # pre_t = pre_t.cpu()
            # pre_rot = pre_rot.cpu()

            # batch_pose0 = torch.from_numpy(pose0).unsqueeze(0).float().to(device)
            # batch_pose1 = torch.from_numpy(pose1).unsqueeze(0).float().to(device)
            # batch_relative_pose = torch.matmul(batch_pose1, batch_pose0.permute(0, 2, 1))
            batch_relative_pose = np.matmul(pose1, np.linalg.inv(pose0))
            batch_relative_pose = torch.from_numpy(batch_relative_pose).unsqueeze(0).float().to(device)
            t_err, R_err = relative_pose_error(batch_relative_pose, pre_rot, pre_t, ignore_gt_t_thr=0.0)

            metrics['t_errs'] = metrics['t_errs'] + np.array(t_err.reshape(-1).cpu().detach().numpy()).tolist()
            metrics['R_errs'] = metrics['R_errs'] + np.array(R_err.reshape(-1).cpu().detach().numpy()).tolist()
            metrics['identifiers'].append(pair_name)

    print(f'Acc: {recall_image}/{all_image}')
    val_metrics_4tb = aggregate_metrics(metrics, 5e-4)
    val_metrics_4tb['AP50'] = recall_image / all_image
    logger.info('\n' + pprint.pformat(val_metrics_4tb))

    res_table.append([f"{name}"] + list(val_metrics_4tb.values()))

2024-01-06 00:11:01.735 | INFO     | __main__:<module>:2 - Onepose: 1/10
100%|██████████| 46/46 [00:00<00:00, 61.39it/s]
2024-01-06 00:11:11.189 | INFO     | __main__:<module>:88 - 
{'AP50': 0.9157175398633257,
 'R:ACC15': 0.6767441860465117,
 'R:ACC30': 0.9837209302325581,
 'R:auc@15': 0.3043439340868661,
 'R:auc@30': 0.5808061372725538,
 'R:medianErr': 10.867824077606201,
 't:ACC15': 0.027906976744186046,
 't:ACC30': 0.15348837209302327,
 't:auc@15': 0.01025364787079567,
 't:auc@30': 0.05103212903636371,
 't:medianErr': 57.68374443054199}
2024-01-06 00:11:11.190 | INFO     | __main__:<module>:2 - Onepose: 2/10


Acc: 402/439


100%|██████████| 34/34 [00:00<00:00, 55.49it/s]
2024-01-06 00:11:16.485 | INFO     | __main__:<module>:88 - 
{'AP50': 0.9149659863945578,
 'R:ACC15': 0.6972789115646258,
 'R:ACC30': 0.9931972789115646,
 'R:auc@15': 0.3344629462816818,
 'R:auc@30': 0.6027458145471115,
 'R:medianErr': 10.290589809417725,
 't:ACC15': 0.04421768707482993,
 't:ACC30': 0.1292517006802721,
 't:auc@15': 0.01186611463153173,
 't:auc@30': 0.05169281072897706,
 't:medianErr': 59.70263671875}
2024-01-06 00:11:16.486 | INFO     | __main__:<module>:2 - Onepose: 3/10


Acc: 269/294


100%|██████████| 30/30 [00:00<00:00, 55.51it/s]
2024-01-06 00:11:20.669 | INFO     | __main__:<module>:88 - 
{'AP50': 0.8986784140969163,
 'R:ACC15': 0.5506607929515418,
 'R:ACC30': 0.9911894273127754,
 'R:auc@15': 0.25215038777097903,
 'R:auc@30': 0.5316885529810812,
 'R:medianErr': 13.477045059204102,
 't:ACC15': 0.03524229074889868,
 't:ACC30': 0.15418502202643172,
 't:auc@15': 0.011481180345084349,
 't:auc@30': 0.054062740113066506,
 't:medianErr': 55.76251983642578}
2024-01-06 00:11:20.669 | INFO     | __main__:<module>:2 - Onepose: 4/10


...

Acc: 258/305


100%|██████████| 34/34 [00:00<00:00, 56.94it/s]
2024-01-06 00:11:52.968 | INFO     | __main__:<module>:88 - 
{'AP50': 0.5535168195718655,
 'R:ACC15': 0.5443425076452599,
 'R:ACC30': 0.9969418960244648,
 'R:auc@15': 0.2600529317583634,
 'R:auc@30': 0.547970008108837,
 'R:medianErr': 13.689202308654785,
 't:ACC15': 0.06422018348623854,
 't:ACC30': 0.1743119266055046,
 't:auc@15': 0.0256067602641721,
 't:auc@30': 0.0697242958465483,
 't:medianErr': 59.28119659423828}
2024-01-06 00:11:52.969 | INFO     | __main__:<module>:2 - Onepose: 10/10


Acc: 181/327


100%|██████████| 32/32 [00:00<00:00, 52.44it/s]
2024-01-06 00:12:00.298 | INFO     | __main__:<module>:88 - 
{'AP50': 0.7475247524752475,
 'R:ACC15': 0.6188118811881188,
 'R:ACC30': 0.9975247524752475,
 'R:auc@15': 0.2848089729598647,
 'R:auc@30': 0.5677398429845426,
 'R:medianErr': 11.823050022125244,
 't:ACC15': 0.04950495049504951,
 't:ACC30': 0.1782178217821782,
 't:auc@15': 0.015998072671418142,
 't:auc@30': 0.059944056874454615,
 't:medianErr': 57.56123924255371}


Acc: 302/404


In [7]:
headers = ["Category"] + list(val_metrics_4tb.keys())
all_data = np.array(res_table)[:, 1:].astype(np.float32)
res_table.append(["Avg"] + all_data.mean(0).tolist())
print(tabulate(res_table, headers=headers, tablefmt='fancy_grid'))

╒════════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤════════════╤════════════╤═══════════╤═══════════╤═══════════════╤══════════╕
│ Category       │   R:auc@15 │   R:auc@30 │   R:ACC15 │   R:ACC30 │   R:medianErr │   t:auc@15 │   t:auc@30 │   t:ACC15 │   t:ACC30 │   t:medianErr │     AP50 │
╞════════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════════╪══════════╡
│ aptamil        │   0.304344 │   0.580806 │  0.676744 │  0.983721 │       10.8678 │  0.0102536 │  0.0510321 │ 0.027907  │  0.153488 │       57.6837 │ 0.915718 │
├────────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼────────────┼────────────┼───────────┼───────────┼───────────────┼──────────┤
│ jzhg           │   0.334463 │   0.602746 │  0.697279 │  0.993197 │       10.2906 │  0.0118661 │  0.0516928 │ 0.0442177 │  0.129252 │       59.7026 │ 0.914966 │
├────────────────┼──────────